<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula05-doc2query/aula05_doc2query_t5_query_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will be used to generate queries for document expansion.

# gcloud authentication

In [ ]:
import os
from google.colab import auth
import tensorflow_gcs_config

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=yCGVbJn0HL8SnrxR9kYEcdYzC3iGxI&prompt=consent&access_type=offline&code_challenge=3B7S5rlDyx9_ZBRpeuHlym1aIGimkVWibVI7EcjsQ98&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk50xn6vi9h6vf2uPl-Z6T8ttUopR92WonqRdulN4KDX3ltsYOXVUNe6peLCB0mPig


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Lj6D5QOUsWNJPsmxu3628vld0k4knC&prompt=consent&access_type=offline&code_challenge=X_LFdba-tbZV4f_qFK31xh03dsj8MHsmx2HFKAvt55U&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk7YoEP4R556h2MVkWxfXXawbAh8aJ9kHpjAlBPNsBzezX_OSsN9TkWv1uNK4RqvFw

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
BUCKET = 'gs://marcospiau-doc2query-aula05'
os.environ['BUCKET'] = BUCKET

In [ ]:
BEST_CKPT = os.path.join(BUCKET, 'models/doc2query_train/model.ckpt-1005516')
os.environ['BEST_CKPT'] = BEST_CKPT

In [ ]:
!gsutil ls $BEST_CKPT.*

gs://marcospiau-doc2query-aula05/models/doc2query_train/model.ckpt-1005516.data-00000-of-00002
gs://marcospiau-doc2query-aula05/models/doc2query_train/model.ckpt-1005516.data-00001-of-00002
gs://marcospiau-doc2query-aula05/models/doc2query_train/model.ckpt-1005516.index
gs://marcospiau-doc2query-aula05/models/doc2query_train/model.ckpt-1005516.meta


# Installs

In [ ]:
%%capture
!pip install -q ftfy polars datasets toolz
!pip install -U t5[gcp,cache-tasks]==0.9.3
!pip install -U jaxlib
!sudo apt install -qq tree htop

# Data preparation

T5 expects a text file containing only text for generation, here process data into this format. Data used for generation will be in the format `f"{ex['title']}. {ex['text']}"`

## Converting each document to a single line

In [ ]:
import datasets
from tqdm import tqdm
import polars as pl
from etils import epath
import re

In [ ]:
f'{title}. {text}'

In [ ]:
def read_hf_concat_title_text() -> pl.DataFrame:
    ds = datasets.load_dataset('BeIR/trec-covid', 'corpus')['corpus']
    df = pl.from_arrow(ds.data.table)
    df = df.with_columns(
        pl.format('{}. {}', 'title', 'text').alias('title_and_text'))

    return df

df = read_hf_concat_title_text()
df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

_id,title,text,title_and_text
str,str,str,str
"""ug7v899j""","""Clinical featu…","""OBJECTIVE: Thi…","""Clinical featu…"
"""02tnwd4m""","""Nitric oxide: …","""Inflammatory d…","""Nitric oxide: …"
"""ejv2xln0""","""Surfactant pro…","""Surfactant pro…","""Surfactant pro…"
"""2b73a28n""","""Role of endoth…","""Endothelin-1 (…","""Role of endoth…"
"""9785vg6d""","""Gene expressio…","""Respiratory sy…","""Gene expressio…"


We have come cases with no text:

In [ ]:
df.filter(~pl.reduce(lambda x, y: x & y, pl.all().str.lengths().ge(1))).head()

_id,title,text,title_and_text
str,str,str,str
"""6iu1dtyl""","""The site of or…","""""","""The site of or…"
"""i5fcedbo""","""Scientific Abs…","""""","""Scientific Abs…"
"""bjjft7ut""","""Society of Gen…","""""","""Society of Gen…"
"""urk7fe34""","""CLINICAL VIGNE…","""""","""CLINICAL VIGNE…"
"""u09rn09u""","""ABSTRACTS""","""""","""ABSTRACTS. """


In [ ]:
def convert_to_t5_format_one_line_per_document(df, base_dir, id_col='_id',
                                               content_col='title_and_text'):
    """Write files with ids and contents. Each document generates only one line"""
    df = df.select(
        pl.col(id_col).alias('id'),
        pl.col(content_col).alias('content')
    )
    # Remove starting #'s as T5 skips those lines by default.
    # segment = re.sub(r'^#*', '', segment)
    df = df.with_columns(pl.col('content').apply(
        lambda x: re.sub(r'^#*', '', x)))
    print(f'Initial length = {len(df)}')
    df = df.drop_nulls()
    print(f'Length after dropping nullss = {len(df)}')
    df = df.filter(pl.reduce(lambda x, y: x & y, pl.all().str.lengths().ge(1)))
    print(f'Length after dropping empty ids or content = {len(df)}')
    epath.Path(base_dir).mkdir(parents=True, exist_ok=True)
    for col in ['id', 'content']:
        df.select(col).write_csv(os.path.join(str(base_dir), f'{col}.txt'),
                                 has_header=False)
    return df



convert_to_t5_format_one_line_per_document(df, 'one_line_per_document')
!head one_line_per_document/*
!ls -lht one_line_per_document
!wc -l one_line_per_document/*

Initial length = 171332
Length after dropping nullss = 171332
Length after dropping empty ids or content = 171332
==> one_line_per_document/content.txt <==
"Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the 

Backup data to GCP:

In [ ]:
!gsutil -m rsync one_line_per_document $BUCKET/ids_and_contents/one_line_per_document

Building synchronization state...
Starting synchronization...
Copying file://one_line_per_document/content.txt [Content-Type=text/plain]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://one_line_per_document/id.txt [Content-Type=text/plain]...
/
Operation completed over 2 objects/185.1 MiB.                                    


In [ ]:
!gsutil ls $BUCKET/ids_and_contents/one_line_per_document

gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt
gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/id.txt


## Convert each document to sliding windows of sentences

Fazer o outro primeiro e ver se dá tempo.

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
tokenizer.batch_decode(model.generate(**tokenizer('#MedStudentCovid: How social media is supporting students during COVID-19. ', return_tensors='pt')))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad><extra_id_0>#MedStudentCovid: How social media is supporting students during COVID']

In [ ]:
re.sub(r'^#*', '', '#SendHelpNow: Mental wellness and virtual connection in the age of coronavirus. ')

'SendHelpNow: Mental wellness and virtual connection in the age of coronavirus. '

In [ ]:
https://github.com/castorini/docTTTTTquery/blob/master/convert_msmarco_doc_to_t5_format.py
# Remove starting #'s as T5 skips those lines by default.
segment = re.sub(r'^#*', '', segment)

In [ ]:
# ANTES
!cat one_line_per_document/content.txt | grep -P '^#'

#BeKind.. 
#MedStudentCovid: How social media is supporting students during COVID-19. 
#SendHelpNow: Mental wellness and virtual connection in the age of coronavirus. 
#pandemicpedagogy: Using Twitter for knowledge exchange. 
#EducationInTheTimeOfCOVID: Leveraging social media to teach during the COVID-19 pandemic pandemonium. 
#GivingTuesdayNow raises over $500 million for COVID‐19 relief. 


In [ ]:
!grep MedStudentCovid one_line_per_document/content.txt

MedStudentCovid: How social media is supporting students during COVID-19. 


In [ ]:
# DEPOIS
!cat one_line_per_document/content.txt | grep -P '^#'

# Generating queries

## Greedy decode

In [ ]:
%%writefile generate_greedy_decode.sh
set -u
MODEL_DIR=$1
CKPT_STEP=$2
IN_FILE=$3
OUT_DIR=$4
LOG_FILE=$5

OUT_FILE="${OUT_DIR}/predictions.txt"

# --gin_file="${MODEL_DIR}/operative_config.gin" \

echo -e "*****pip requirements*****\n$(pip freeze)" > $LOG_FILE
echo Start time: $(date) >> $LOG_FILE

(time python3 -m t5.models.mesh_transformer_main  \
--tpu="${TPU_NAME}" \
--model_dir="${MODEL_DIR}" \
--gin_file="gs://t5-data/pretrained_models/base/operative_config.gin" \
--gin_param="infer_checkpoint_step = ${CKPT_STEP}" \
--gin_param="input_filename = '${IN_FILE}'" \
--gin_param="output_filename = '${OUT_FILE}'" \
--gin_param="utils.tpu_mesh_shape.tpu_topology ='v2-8'" \
--gin_param="tokens_per_batch = 131072" \
--gin_file="infer.gin" \
--gin_param="utils.run.sequence_length = {'inputs': 512, 'targets': 64}" \
--gin_param="Bitransformer.decode.max_decode_length = 64" \
--gin_file="greedy_decode.gin" \
$@ ) 2>&1 | tee -a $LOG_FILE

echo Finish time: $(date) >> $LOG_FILE
FINAL_LOG_FILE=${OUT_DIR}/$(basename $LOG_FILE)
echo Final log file is $FINAL_LOG_FILE
gsutil cp $LOG_FILE $FINAL_LOG_FILE

Overwriting generate_greedy_decode.sh


In [ ]:
!chmod +x generate_greedy_decode.sh

In [ ]:
! ./generate_greedy_decode.sh \
gs://marcospiau-doc2query-aula05/models/doc2query_train \
1005516 \
gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt \
$BUCKET/generated_queries/one_line_per_document_greedy_decode \
one_line_per_document_greedy_decode.logs

./generate_greedy_decode.sh: line 11: --gin_file=gs://marcospiau-doc2query-aula05/models/doc2query_train/operative_config.gin: No such file or directory
Start time: Sun 09 Apr 2023 07:32:08 AM UTC
2023-04-09 07:32:10.461166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
I0409 07:32:16.538161 140247151294272 resource_reader.py:50] system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
E0409 07:32:16.538861 140247151294272 resource_reader.py:55] Path not found: gs://t5-data/pretrained_models/base/operative_config.gin
I0409 07:32:16.722113 140247151294272 config.py:2372] Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).
I0409 07:32:16.739379 140247151294272 resource_reader.py:50] system_path_file_exists:infer.gin
E0409 07:32:16.739714 140247151294272 resource_reader.py:55] Path not found: in

In [ ]:
!gsutil cat gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/predictions.txt-1005516 | head 

how many patients died from m pneumoniae
what is NO• in the lung
what is surfactant protein d
what is endothelin-1
what are the effects of pneumovirus on the respiratory epithelial cells?
which sequence is required for a discontinuous rna synthesis
does transfusing to normal haemoglobin enhance outcome
what was the theme of the international intensive care symposium
how does heme oxygenase-1 work in pulmonary disease
what is a rods outbreak


In [ ]:
!gsutil du -ahc $BUCKET/generated_queries/one_line_per_document_greedy_decode

412.2 KiB    gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/one_line_per_document_greedy_decode.log#1681026281942471
6.88 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/predictions.txt-1005516#1681026278561663
7.29 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_greedy_decode/
7.29 MiB     total


## Beam search

In [ ]:
%%writefile generate_beam_search.sh
set -u
MODEL_DIR=$1
CKPT_STEP=$2
IN_FILE=$3
OUT_DIR=$4
LOG_FILE=$5

OUT_FILE="${OUT_DIR}/predictions.txt"

# --gin_file="${MODEL_DIR}/operative_config.gin" \

echo -e "*****pip requirements*****\n$(pip freeze)" > $LOG_FILE
echo Start time: $(date) >> $LOG_FILE

(time python3 -m t5.models.mesh_transformer_main  \
--tpu="${TPU_NAME}" \
--model_dir="${MODEL_DIR}" \
--gin_file="gs://t5-data/pretrained_models/base/operative_config.gin" \
--gin_param="infer_checkpoint_step = ${CKPT_STEP}" \
--gin_param="input_filename = '${IN_FILE}'" \
--gin_param="output_filename = '${OUT_FILE}'" \
--gin_param="utils.tpu_mesh_shape.tpu_topology ='v2-8'" \
--gin_param="tokens_per_batch = 131072" \
--gin_file="infer.gin" \
--gin_param="utils.run.sequence_length = {'inputs': 512, 'targets': 64}" \
--gin_param="Bitransformer.decode.max_decode_length = 64" \
--gin_file="beam_search.gin" \
$@ ) 2>&1 | tee -a $LOG_FILE

echo Finish time: $(date) >> $LOG_FILE
FINAL_LOG_FILE=${OUT_DIR}/$(basename $LOG_FILE)
echo Final log file is $FINAL_LOG_FILE
gsutil cp $LOG_FILE $FINAL_LOG_FILE

Overwriting generate_beam_search.sh


In [ ]:
!chmod +x generate_beam_search.sh
! ./generate_beam_search.sh \
gs://marcospiau-doc2query-aula05/models/doc2query_train \
1005516 \
gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt \
$BUCKET/generated_queries/one_line_per_document_beam_search \
one_line_per_document_beam_search.log

2023-04-09 07:48:52.267783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
I0409 07:48:58.419850 139931331848000 resource_reader.py:50] system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
E0409 07:48:58.420622 139931331848000 resource_reader.py:55] Path not found: gs://t5-data/pretrained_models/base/operative_config.gin
I0409 07:48:58.522869 139931331848000 config.py:2372] Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).
I0409 07:48:58.539582 139931331848000 resource_reader.py:50] system_path_file_exists:infer.gin
E0409 07:48:58.539922 139931331848000 resource_reader.py:55] Path not found: infer.gin
I0409 07:48:58.541093 139931331848000 resource_reader.py:50] system_path_file_exists:beam_search.gin
E0409 07:48:58.541298 139931331848000 resource_reader.py:55] Path not found: beam_search

In [ ]:
!gsutil cat gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/predictions.txt-1005516 | heads

how many patients died from m pneumoniae
what is NO• in the lung
what is surfactant protein d
what is endothelin-1
what are the effects of pneumovirus on the respiratory epithelial cells?
which sequence is required for strand transfer during nidovirus synthesis
does transfusing to normal haemoglobin enhance outcome
what was the theme of the international intensive care symposium
how does heme oxygenase-1 work in pulmonary disease
what is a rods system


Generated files:

In [ ]:
!gsutil du -ahc $BUCKET/generated_queries/one_line_per_document_beam_search

0 B          gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/#1681027254873984
412.46 KiB   gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/one_line_per_document_beam_search.log#1681027258969118
6.79 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/predictions.txt-1005516#1681027255475607
7.2 MiB      gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_beam_search/
7.2 MiB      total


## Top-k sampling

In [ ]:
%%writefile generate_topk_sampling.sh
set -u
MODEL_DIR=$1
CKPT_STEP=$2
IN_FILE=$3
OUT_DIR=$4
LOG_FILE=$5
N_ITER=$6


echo -e "*****pip requirements*****\n$(pip freeze)" > $LOG_FILE
echo Will run $N_ITER iterations >> $LOG_FILE
echo Will run $N_ITER iterations

for i in $(seq -f %02g 1 $N_ITER);
do
    echo "Iteration $i" >> $LOG_FILE
    # check if file already exists
    OUT_FILE="${OUT_DIR}/predictions-iter-${i}.txt"
    if gsutil -q stat ${OUT_FILE}-${CKPT_STEP}; then
        echo "File $OUT_FILE already exists. Skipping." >> $LOG_FILE
        echo "File $OUT_FILE already exists. Skipping."
        continue
    fi
    echo "File $OUT_FILE does not exist. Generating." >> $LOG_FILE
    echo Start time iter $i: $(date) >> $LOG_FILE
    (time python3 -m t5.models.mesh_transformer_main  \
    --tpu="${TPU_NAME}" \
    --model_dir="${MODEL_DIR}" \
    --gin_file="gs://t5-data/pretrained_models/base/operative_config.gin" \
    --gin_param="infer_checkpoint_step = ${CKPT_STEP}" \
    --gin_param="input_filename = '${IN_FILE}'" \
    --gin_param="output_filename = '${OUT_FILE}'" \
    --gin_param="utils.tpu_mesh_shape.tpu_topology ='v2-8'" \
    --gin_param="tokens_per_batch = 131072" \
    --gin_file="infer.gin" \
    --gin_param="utils.run.sequence_length = {'inputs': 512, 'targets': 64}" \
    --gin_param="Bitransformer.decode.max_decode_length = 64" \
    --gin_file="sample_decode.gin" \
    --gin_param="Bitransformer.decode.temperature = 1.0" \
    --gin_param="Unitransformer.sample_autoregressive.sampling_keep_top_k = 10" \
    ) >> $LOG_FILE 2>&1 
    echo End time iter $i: $(date) >> $LOG_FILE
done

# ) 2>&1 | tee -a $LOG_FILE

echo Finish time: $(date) >> $LOG_FILE
FINAL_LOG_FILE=${OUT_DIR}/$(basename $LOG_FILE)
echo Final log file is $FINAL_LOG_FILE
gsutil cp $LOG_FILE $FINAL_LOG_FILE

Overwriting generate_topk_sampling.sh


In [ ]:
!chmod +x generate_topk_sampling.sh
! ./generate_topk_sampling.sh \
gs://marcospiau-doc2query-aula05/models/doc2query_train \
1005516 \
gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt \
$BUCKET/generated_queries/one_line_per_document_topk_sampling \
one_line_per_document_topk_sampling.log \
20

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0409 19:38:12.274480 140615591675712 tpu_estimator.py:603] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0409 19:38:12.274689 140615591675712 tpu_estimator.py:607] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0409 19:38:13.029023 140615591675712 tpu_estimator.py:603] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0409 19:38:13.029237 140615591675712 tpu_estimator.py:607] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0409 19:38:13.735949 140615591675712 tpu_estimator.py:603] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0409 19:38:13.736161 1406155916

In [16]:
!chmod +x generate_topk_sampling.sh
! ./generate_topk_sampling.sh \
gs://marcospiau-doc2query-aula05/models/doc2query_train \
1005516 \
gs://marcospiau-doc2query-aula05/ids_and_contents/one_line_per_document/content.txt \
$BUCKET/generated_queries/one_line_per_document_topk_sampling \
one_line_per_document_topk_sampling.log \
60

Will run 60 iterations
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-01.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-02.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-03.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-04.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-05.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-06.txt already exists. Skipping.
File gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-07.txt already e

Generated files:

In [19]:
!gsutil du -ahc $BUCKET/generated_queries/one_line_per_document_topk_sampling

7.9 MiB      gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/one_line_per_document_topk_sampling.log#1681340871754416
6.95 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-01.txt-1005516#1681056148126254
6.95 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-02.txt-1005516#1681056943246145
6.95 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-03.txt-1005516#1681057664655908
6.95 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-04.txt-1005516#1681058391346997
6.96 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_sampling/predictions-iter-05.txt-1005516#1681059113098832
6.96 MiB     gs://marcospiau-doc2query-aula05/generated_queries/one_line_per_document_topk_samplin